In [6]:
from raphtory import Graph

device_graph = Graph()

with open("data/devices.csv", 'r') as device_file:
    next(device_file) # skip the header
    for device in device_file:
        device = device.split(",")
        neId = device[0]
        code = device[1]
        name = device[2]
        lat = float(device[3])
        long = float(device[4])
        properties = {"code":code,"name":name,"lat":lat,"long":long}
        device_graph.add_vertex(timestamp=1,id=neId,properties=properties)

with open("data/links.csv", 'r') as links_file:
    next(links_file) # skip the header
    for link in links_file:
        link = link.split(",")
        src=link[0]
        dst=link[1]
        link_id=link[2]
        capacity=int(link[3])
        properties = {"link_id":link_id,"capacity":capacity}
        device_graph.add_edge(timestamp=1,src=src,dst=dst,properties=properties)

device_graph

Graph(number_of_edges=29, number_of_vertices=15, earliest_time=1, latest_time=1)